In [16]:
import numpy              as np
import seaborn            as sns
import matplotlib.pyplot  as plt
import networkx           as nx
import random
import time
import datetime
import os

from   experiments.indiv_u_side_exps_full import indiv_u_side_multiple_exp
from   experiments.indiv_u_side_exps_full import indiv_u_side_exp4

from   experiments.group_group_exps_full  import remove_best_k
# from   experiments.group_level_experiments      import group_experiment_3

import utils.synthetic_gen as sg

from   utils.recwalk_funcs              import create_recwalk_model, create_recwalk_model_from_adj
from   utils.counterfactual             import ppr_absorbing_3
from   model.user_item_dataset          import UserItemDataset

In [2]:
# enable inline plotting
%matplotlib inline
%config InlineBackend.figure_format = 'svg' # set format to svg for better resolution
sns.set_theme() # apply default theme in seaborn

## Matrix inversion to produce $p_x$ scores for every node x

In [3]:
def calculate_matrix_Q(P, gamma=0.15):
    return gamma*np.linalg.inv((np.identity(P.shape[0]) - (1-gamma)*P))

## Synthetic dataset generation

---

* Generate and export datasets with different variables
* **Size**:
    * num_users = 1000
    * num_items = 1000
* **Population:**
    * r_u = 0.5 (default)
    * r_i = 0.5 (default)
* **Popularity:**
    * zipf_a_0 = 1.3 (default)
    * zipf_a_1 = 1.3 (default)
    * Other values: (1, 1.1, 1.5, 1.7)
* **Bias:**
    * p00 = 0.7 (default)
    * p11 = 0.7 (default)
    * Other values: (0.5, 0.6, 0.8, 0.9)

In [3]:
num_users, num_items = 1000, 1000
r_u, r_i = 0.5, 0.5
bias_values = [0.5, 0.6, 0.7, 0.8, 0.9] # Probabilities of p00 and p11 (symmetric)
popularity_values = [1, 1.1, 1.3, 1.5, 1.7]
ratings_per_user = num_items // 10 # 10% of the items
num_initial_ratings = 5

export_path='../datasets/synthetic'

# Create various datasets for different parameters
st = time.time()
for i in range(len(bias_values)):
    for j in range(len(popularity_values)):
        cur_bias, cur_pop = bias_values[i], popularity_values[j]
        
        export_filename = f"synth_{cur_bias}_b_{cur_pop}_p"
        
        print(f"[{datetime.datetime.now()}] Generation of dataset with bias={cur_bias} and popularity={cur_pop} started..")
        sg.generate_synthetic_graph(num_users=num_users,
                                    num_items=num_items,
                                    r_u=r_u,
                                    r_i=r_i,
                                    p00=cur_bias,
                                    p11=cur_bias,
                                    ratings_per_user=ratings_per_user,
                                    zipf_a_0=cur_pop,
                                    zipf_a_1=cur_pop,
                                    k=num_initial_ratings,
                                    export_to_file = True,
                                    path=export_path,
                                    filename=export_filename)
        print(f"[{datetime.datetime.now()}] Dataset generation completed..")
        print('-------')
print(f"Overall generation time: {round((time.time() - st)/60, 4)} minutes")

[2023-03-19 22:14:46.182703] Generation of dataset with bias=0.5 and popularity=1 started..
[2023-03-19 22:14:49.024307] Dataset generation completed..
-------
[2023-03-19 22:14:49.024307] Generation of dataset with bias=0.5 and popularity=1.1 started..
[2023-03-19 22:14:52.573551] Dataset generation completed..
-------
[2023-03-19 22:14:52.574551] Generation of dataset with bias=0.5 and popularity=1.3 started..
[2023-03-19 22:14:56.281083] Dataset generation completed..
-------
[2023-03-19 22:14:56.281083] Generation of dataset with bias=0.5 and popularity=1.5 started..
[2023-03-19 22:14:58.244552] Dataset generation completed..
-------
[2023-03-19 22:14:58.244552] Generation of dataset with bias=0.5 and popularity=1.7 started..
[2023-03-19 22:15:00.926090] Dataset generation completed..
-------
[2023-03-19 22:15:00.927090] Generation of dataset with bias=0.6 and popularity=1 started..
[2023-03-19 22:15:03.633383] Dataset generation completed..
-------
[2023-03-19 22:15:03.634384] Gen

### Produce and export random sample of users for every synthetic dataset

In [3]:
sample_size = 50
users_num = 1000

num_of_synthetic_datasets = 30

# Export random sample to always use the same random sequence
export_dir_path = './output/users_sample'
# Produce a sequence number to avoid overwriting an existing random sample by accident
index_num = len(os.listdir(export_dir_path)) + 1
export_filename = f"random_sample_s_{sample_size}_all_{index_num}"

with open(f"{export_dir_path}/{export_filename}.tsv", 'w') as f:
    for i in range(num_of_synthetic_datasets):
        random_sample = np.array(random.sample(list(range(users_num)), k=sample_size))
        for u in random_sample:
            if u != random_sample[-1]:
                f.write(f"{u}\t")
            else:
                f.write(f"{u}")
        if i != (num_of_synthetic_datasets - 1):
            f.write('\n')

## Individual-level - Experiment 3

---

In [7]:
sample_size = 20
genre_absorbing_col = 1
directions_removed = 1
nbs_to_remove_limit = 100

export_dir_path = './output/full/individual_user_side/exp3/males'

synth_dir_path = '../datasets/synthetic'
synth_filenames = [filename for filename in os.listdir(synth_dir_path) if filename.split('.')[-1] == 'edges']
for s in synth_filenames:
    filename_no_ext = '.'.join(s.split('.')[:-1])

    info_dict, R = sg.read_graph(synth_dir_path, '.'.join(s.split('.')[:-1])) # Read synthetic dataset
    
    users_num, items_num = info_dict['num_users'], info_dict['num_items']
    num_items_0 = round(info_dict['r_i']*items_num)
    users_num_0 = round(info_dict['r_u']*users_num)
    user_original_ids, item_original_ids = np.arange(users_num), np.arange(items_num)
    item_group_mapped_ids_0, item_group_mapped_ids_1 = np.arange(num_items_0), np.arange(num_items_0, items_num)
    
    rng = np.random.default_rng(seed=np.random.randint(10000)) # Give a seed to produce same sample
    sample_users_random = rng.choice(user_original_ids[:users_num_0],
                                     size=sample_size,
                                     replace=False)
    
    export_filename = f'exp3_{filename_no_ext}_{sample_size}_{nbs_to_remove_limit}'
    
    indiv_u_side_multiple_exp(R                       = R,
                              user_original_ids       = user_original_ids,
                              item_original_ids       = item_original_ids,
                              item_group_mapped_ids_0 = item_group_mapped_ids_0,
                              item_group_mapped_ids_1 = item_group_mapped_ids_1,
                              genre_absorbing_col     = genre_absorbing_col,
                              directions_removed      = directions_removed,
                              sample_users            = sample_users_random,
                              nbs_to_remove_limit     = nbs_to_remove_limit,
                              export_dir_path         = export_dir_path,
                              export_filename         = export_filename)

[2023-04-01 18:39:17.147960] User 1 - id: 396:#neighbors=100
[2023-04-01 18:39:21.524518] 10 neighbors removed (refreshed per 10)
[2023-04-01 18:39:21.900306] 20 neighbors removed (refreshed per 10)
[2023-04-01 18:39:22.274094] 30 neighbors removed (refreshed per 10)
[2023-04-01 18:39:22.541909] 40 neighbors removed (refreshed per 10)
[2023-04-01 18:39:22.769787] 50 neighbors removed (refreshed per 10)
[2023-04-01 18:39:22.974663] 60 neighbors removed (refreshed per 10)
[2023-04-01 18:39:23.174559] 70 neighbors removed (refreshed per 10)
[2023-04-01 18:39:23.298515] 80 neighbors removed (refreshed per 10)
[2023-04-01 18:39:23.358482] 90 neighbors removed (refreshed per 10)
----------
[2023-04-01 18:39:23.380469] User 2 - id: 97:#neighbors=100
[2023-04-01 18:39:27.550111] 10 neighbors removed (refreshed per 10)
[2023-04-01 18:39:27.907908] 20 neighbors removed (refreshed per 10)
[2023-04-01 18:39:28.281697] 30 neighbors removed (refreshed per 10)
[2023-04-01 18:39:28.550545] 40 neighbor

[2023-04-01 18:40:37.387583] 10 neighbors removed (refreshed per 10)
[2023-04-01 18:40:37.846324] 20 neighbors removed (refreshed per 10)
[2023-04-01 18:40:38.357026] 30 neighbors removed (refreshed per 10)
[2023-04-01 18:40:38.751838] 40 neighbors removed (refreshed per 10)
[2023-04-01 18:40:39.036642] 50 neighbors removed (refreshed per 10)
[2023-04-01 18:40:39.338471] 60 neighbors removed (refreshed per 10)
[2023-04-01 18:40:39.603323] 70 neighbors removed (refreshed per 10)
[2023-04-01 18:40:39.757234] 80 neighbors removed (refreshed per 10)
[2023-04-01 18:40:39.874175] 90 neighbors removed (refreshed per 10)
----------
[2023-04-01 18:40:39.938140] User 14 - id: 371:#neighbors=100
[2023-04-01 18:40:44.522574] 10 neighbors removed (refreshed per 10)
[2023-04-01 18:40:44.887333] 20 neighbors removed (refreshed per 10)
[2023-04-01 18:40:45.234172] 30 neighbors removed (refreshed per 10)
[2023-04-01 18:40:45.529970] 40 neighbors removed (refreshed per 10)
[2023-04-01 18:40:45.979725] 5

[2023-04-01 18:41:52.195924] 10 neighbors removed (refreshed per 10)
[2023-04-01 18:41:52.536758] 20 neighbors removed (refreshed per 10)
[2023-04-01 18:41:52.858577] 30 neighbors removed (refreshed per 10)
[2023-04-01 18:41:53.159371] 40 neighbors removed (refreshed per 10)
[2023-04-01 18:41:53.401244] 50 neighbors removed (refreshed per 10)
[2023-04-01 18:41:53.584166] 60 neighbors removed (refreshed per 10)
[2023-04-01 18:41:53.741042] 70 neighbors removed (refreshed per 10)
[2023-04-01 18:41:53.846018] 80 neighbors removed (refreshed per 10)
[2023-04-01 18:41:53.904985] 90 neighbors removed (refreshed per 10)
----------
[2023-04-01 18:41:53.926946] User 6 - id: 211:#neighbors=100
[2023-04-01 18:41:58.091616] 10 neighbors removed (refreshed per 10)
[2023-04-01 18:41:58.476363] 20 neighbors removed (refreshed per 10)
[2023-04-01 18:41:58.790186] 30 neighbors removed (refreshed per 10)
[2023-04-01 18:41:59.089017] 40 neighbors removed (refreshed per 10)
[2023-04-01 18:41:59.344881] 50

[2023-04-01 18:43:07.060890] 10 neighbors removed (refreshed per 10)
[2023-04-01 18:43:07.425676] 20 neighbors removed (refreshed per 10)
[2023-04-01 18:43:07.766482] 30 neighbors removed (refreshed per 10)
[2023-04-01 18:43:08.087336] 40 neighbors removed (refreshed per 10)
[2023-04-01 18:43:08.371148] 50 neighbors removed (refreshed per 10)
[2023-04-01 18:43:08.594016] 60 neighbors removed (refreshed per 10)
[2023-04-01 18:43:08.753959] 70 neighbors removed (refreshed per 10)
[2023-04-01 18:43:08.876889] 80 neighbors removed (refreshed per 10)
[2023-04-01 18:43:08.948848] 90 neighbors removed (refreshed per 10)
----------
[2023-04-01 18:43:08.971809] User 18 - id: 47:#neighbors=100
[2023-04-01 18:43:13.760091] 10 neighbors removed (refreshed per 10)
[2023-04-01 18:43:14.124893] 20 neighbors removed (refreshed per 10)
[2023-04-01 18:43:14.448702] 30 neighbors removed (refreshed per 10)
[2023-04-01 18:43:14.723547] 40 neighbors removed (refreshed per 10)
[2023-04-01 18:43:14.943425] 50

[2023-04-01 18:44:21.495044] 10 neighbors removed (refreshed per 10)
[2023-04-01 18:44:21.876821] 20 neighbors removed (refreshed per 10)
[2023-04-01 18:44:22.195676] 30 neighbors removed (refreshed per 10)
[2023-04-01 18:44:22.477489] 40 neighbors removed (refreshed per 10)
[2023-04-01 18:44:22.689367] 50 neighbors removed (refreshed per 10)
[2023-04-01 18:44:22.887249] 60 neighbors removed (refreshed per 10)
[2023-04-01 18:44:23.035165] 70 neighbors removed (refreshed per 10)
[2023-04-01 18:44:23.150101] 80 neighbors removed (refreshed per 10)
[2023-04-01 18:44:23.212101] 90 neighbors removed (refreshed per 10)
----------
[2023-04-01 18:44:23.234089] User 10 - id: 336:#neighbors=100
[2023-04-01 18:44:27.384705] 10 neighbors removed (refreshed per 10)
[2023-04-01 18:44:27.723550] 20 neighbors removed (refreshed per 10)
[2023-04-01 18:44:28.029340] 30 neighbors removed (refreshed per 10)
[2023-04-01 18:44:28.359190] 40 neighbors removed (refreshed per 10)
[2023-04-01 18:44:28.622019] 5

...

[2023-04-01 19:23:56.629925] 80 neighbors removed (refreshed per 10)
[2023-04-01 19:23:56.690919] 90 neighbors removed (refreshed per 10)
----------
[2023-04-01 19:23:56.712872] User 4 - id: 64:#neighbors=100
[2023-04-01 19:24:00.864343] 10 neighbors removed (refreshed per 10)
[2023-04-01 19:24:01.236106] 20 neighbors removed (refreshed per 10)
[2023-04-01 19:24:01.545958] 30 neighbors removed (refreshed per 10)
[2023-04-01 19:24:01.832762] 40 neighbors removed (refreshed per 10)
[2023-04-01 19:24:02.111602] 50 neighbors removed (refreshed per 10)
[2023-04-01 19:24:02.285504] 60 neighbors removed (refreshed per 10)
[2023-04-01 19:24:02.435455] 70 neighbors removed (refreshed per 10)
[2023-04-01 19:24:02.533363] 80 neighbors removed (refreshed per 10)
[2023-04-01 19:24:02.593733] 90 neighbors removed (refreshed per 10)
----------
[2023-04-01 19:24:02.615720] User 5 - id: 322:#neighbors=100
[2023-04-01 19:24:08.456940] 10 neighbors removed (refreshed per 10)
[2023-04-01 19:24:08.792118] 

[2023-04-01 19:25:09.159231] 70 neighbors removed (refreshed per 10)
[2023-04-01 19:25:09.293163] 80 neighbors removed (refreshed per 10)
[2023-04-01 19:25:09.364124] 90 neighbors removed (refreshed per 10)
----------
[2023-04-01 19:25:09.390138] User 16 - id: 32:#neighbors=100
[2023-04-01 19:25:13.481067] 10 neighbors removed (refreshed per 10)
[2023-04-01 19:25:13.811305] 20 neighbors removed (refreshed per 10)
[2023-04-01 19:25:14.095145] 30 neighbors removed (refreshed per 10)
[2023-04-01 19:25:14.343040] 40 neighbors removed (refreshed per 10)
[2023-04-01 19:25:14.551889] 50 neighbors removed (refreshed per 10)
[2023-04-01 19:25:14.734192] 60 neighbors removed (refreshed per 10)
[2023-04-01 19:25:14.870150] 70 neighbors removed (refreshed per 10)
[2023-04-01 19:25:14.966096] 80 neighbors removed (refreshed per 10)
[2023-04-01 19:25:15.025063] 90 neighbors removed (refreshed per 10)
----------
[2023-04-01 19:25:15.047050] User 17 - id: 466:#neighbors=100
[2023-04-01 19:25:18.960576

[2023-04-01 19:26:26.786183] 50 neighbors removed (refreshed per 10)
[2023-04-01 19:26:26.985070] 60 neighbors removed (refreshed per 10)
[2023-04-01 19:26:27.135979] 70 neighbors removed (refreshed per 10)
[2023-04-01 19:26:27.237927] 80 neighbors removed (refreshed per 10)
[2023-04-01 19:26:27.296896] 90 neighbors removed (refreshed per 10)
----------
[2023-04-01 19:26:27.319884] User 8 - id: 84:#neighbors=100
[2023-04-01 19:26:32.031601] 10 neighbors removed (refreshed per 10)
[2023-04-01 19:26:32.527292] 20 neighbors removed (refreshed per 10)
[2023-04-01 19:26:32.812542] 30 neighbors removed (refreshed per 10)
[2023-04-01 19:26:33.093393] 40 neighbors removed (refreshed per 10)
[2023-04-01 19:26:33.592108] 50 neighbors removed (refreshed per 10)
[2023-04-01 19:26:33.890941] 60 neighbors removed (refreshed per 10)
[2023-04-01 19:26:34.025856] 70 neighbors removed (refreshed per 10)
[2023-04-01 19:26:34.126835] 80 neighbors removed (refreshed per 10)
[2023-04-01 19:26:34.300740] 90 

[2023-04-01 19:27:41.922558] 50 neighbors removed (refreshed per 10)
[2023-04-01 19:27:42.027500] 60 neighbors removed (refreshed per 10)
[2023-04-01 19:27:42.103461] 70 neighbors removed (refreshed per 10)
[2023-04-01 19:27:42.184410] 80 neighbors removed (refreshed per 10)
[2023-04-01 19:27:42.229388] 90 neighbors removed (refreshed per 10)
----------
[2023-04-01 19:27:42.243380] User 20 - id: 415:#neighbors=100
[2023-04-01 19:27:44.168761] 10 neighbors removed (refreshed per 10)
[2023-04-01 19:27:44.314682] 20 neighbors removed (refreshed per 10)
[2023-04-01 19:27:44.514604] 30 neighbors removed (refreshed per 10)
[2023-04-01 19:27:44.788829] 40 neighbors removed (refreshed per 10)
[2023-04-01 19:27:45.030691] 50 neighbors removed (refreshed per 10)
[2023-04-01 19:27:45.214588] 60 neighbors removed (refreshed per 10)
[2023-04-01 19:27:45.386491] 70 neighbors removed (refreshed per 10)
[2023-04-01 19:27:45.485470] 80 neighbors removed (refreshed per 10)
[2023-04-01 19:27:45.546400] 9

[2023-04-01 19:28:49.143268] 30 neighbors removed (refreshed per 10)
[2023-04-01 19:28:49.410153] 40 neighbors removed (refreshed per 10)
[2023-04-01 19:28:49.618366] 50 neighbors removed (refreshed per 10)
[2023-04-01 19:28:49.827256] 60 neighbors removed (refreshed per 10)
[2023-04-01 19:28:49.970167] 70 neighbors removed (refreshed per 10)
[2023-04-01 19:28:50.066113] 80 neighbors removed (refreshed per 10)
[2023-04-01 19:28:50.126079] 90 neighbors removed (refreshed per 10)
----------
[2023-04-01 19:28:50.148067] User 12 - id: 72:#neighbors=100
[2023-04-01 19:28:54.189570] 10 neighbors removed (refreshed per 10)
[2023-04-01 19:28:54.545377] 20 neighbors removed (refreshed per 10)
[2023-04-01 19:28:54.851623] 30 neighbors removed (refreshed per 10)
[2023-04-01 19:28:55.102510] 40 neighbors removed (refreshed per 10)
[2023-04-01 19:28:55.323385] 50 neighbors removed (refreshed per 10)
[2023-04-01 19:28:55.498285] 60 neighbors removed (refreshed per 10)
[2023-04-01 19:28:55.636586] 70

## Experiment 4: Study item properties

In [12]:
sample_size = 200
genre_absorbing_col = 1
export_dir_path = './output/full/individual_user_side/exp4'

synth_dir_path = '../datasets/synthetic'
synth_filename = 'synth_0.7_b_1.3_p'

export_filename = f"exp4_{sample_size}_random_{synth_filename}"

# Read synthetic dataset
info_dict, R = sg.read_graph(synth_dir_path, synth_filename)

users_num, items_num = info_dict['num_users'], info_dict['num_items']
num_items_0 = round(info_dict['r_i']*items_num)
users_num_0 = round(info_dict['r_u']*users_num)
user_original_ids, item_original_ids = np.arange(users_num), np.arange(items_num)
item_group_mapped_ids_0, item_group_mapped_ids_1 = np.arange(num_items_0), np.arange(num_items_0, items_num)

# Generate random sample
rng = np.random.default_rng(seed=1)
sample_users_random = rng.choice(user_original_ids, size=sample_size, replace=False)

indiv_u_side_exp4(R                       = R,
                  user_original_ids       = user_original_ids,
                  item_original_ids       = item_original_ids,
                  item_group_mapped_ids_0 = item_group_mapped_ids_0,
                  item_group_mapped_ids_1 = item_group_mapped_ids_1,
                  genre_absorbing_col     = genre_absorbing_col,
                  sample_users            = sample_users_random,
                  export_dir_path         = export_dir_path,
                  export_filename         = export_filename)

[2023-03-20 14:23:29.428974] User 1 - id: 617:#neighbors=100
[2023-03-20 14:23:33.473629] User 2 - id: 877:#neighbors=100
[2023-03-20 14:23:37.376386] User 3 - id: 552:#neighbors=100
[2023-03-20 14:23:41.546034] User 4 - id: 462:#neighbors=100
[2023-03-20 14:23:45.618506] User 5 - id: 598:#neighbors=100
[2023-03-20 14:23:49.643236] User 6 - id: 549:#neighbors=100
[2023-03-20 14:23:53.726383] User 7 - id: 606:#neighbors=100
[2023-03-20 14:23:58.073886] User 8 - id: 764:#neighbors=100
[2023-03-20 14:24:00.628422] User 9 - id: 668:#neighbors=100
[2023-03-20 14:24:04.584192] User 10 - id: 100:#neighbors=100
[2023-03-20 14:24:08.311059] User 11 - id: 494:#neighbors=100
[2023-03-20 14:24:12.358740] User 12 - id: 455:#neighbors=100
[2023-03-20 14:24:16.241087] User 13 - id: 305:#neighbors=100
[2023-03-20 14:24:20.194823] User 14 - id: 673:#neighbors=100
[2023-03-20 14:24:24.534302] User 15 - id: 23:#neighbors=100
[2023-03-20 14:24:28.467086] User 16 - id: 466:#neighbors=100
[2023-03-20 14:24:

[2023-03-20 14:31:58.152260] User 133 - id: 711:#neighbors=100
[2023-03-20 14:32:00.991666] User 134 - id: 285:#neighbors=100
[2023-03-20 14:32:04.935372] User 135 - id: 909:#neighbors=100
[2023-03-20 14:32:08.659241] User 136 - id: 70:#neighbors=100
[2023-03-20 14:32:12.334170] User 137 - id: 54:#neighbors=100
[2023-03-20 14:32:16.316852] User 138 - id: 234:#neighbors=100
[2023-03-20 14:32:19.204207] User 139 - id: 236:#neighbors=100
[2023-03-20 14:32:23.022021] User 140 - id: 725:#neighbors=100
[2023-03-20 14:32:26.833838] User 141 - id: 30:#neighbors=100
[2023-03-20 14:32:30.919491] User 142 - id: 272:#neighbors=100
[2023-03-20 14:32:35.015145] User 143 - id: 16:#neighbors=100
[2023-03-20 14:32:38.860979] User 144 - id: 277:#neighbors=100
[2023-03-20 14:32:42.645783] User 145 - id: 60:#neighbors=100
[2023-03-20 14:32:46.487575] User 146 - id: 664:#neighbors=100
[2023-03-20 14:32:50.687170] User 147 - id: 417:#neighbors=100
[2023-03-20 14:32:54.971717] User 148 - id: 204:#neighbors=1

## Experiment 5

Remove best *nbs_to_remove* neighbors to calculate correlation between rank of removal and popularity.

In [4]:
sample_size = 200
genre_absorbing_col = 1
directions_removed = 1
nbs_to_remove = 10
export_dir_path = './output/full/individual_user_side/exp5'

synth_dir_path = '../datasets/synthetic'
synth_filename = 'synth_0.7_b_1.3_p'

export_filename = f"exp5_{sample_size}_random_best_{nbs_to_remove}_{synth_filename}"

# Read synthetic dataset
info_dict, R = sg.read_graph(synth_dir_path, synth_filename)

users_num, items_num = info_dict['num_users'], info_dict['num_items']
num_items_0 = round(info_dict['r_i']*items_num)
users_num_0 = round(info_dict['r_u']*users_num)
user_original_ids, item_original_ids = np.arange(users_num), np.arange(items_num)
item_group_mapped_ids_0, item_group_mapped_ids_1 = np.arange(num_items_0), np.arange(num_items_0, items_num)

# Generate random sample
rng = np.random.default_rng(seed=1)
sample_users_random = rng.choice(user_original_ids, size=sample_size, replace=False)

indiv_u_side_multiple_exp(R                       = R,
                          user_original_ids       = user_original_ids,
                          item_original_ids       = item_original_ids,
                          item_group_mapped_ids_0 = item_group_mapped_ids_0,
                          item_group_mapped_ids_1 = item_group_mapped_ids_1,
                          genre_absorbing_col     = genre_absorbing_col,
                          directions_removed      = directions_removed,
                          sample_users            = sample_users_random,
                          nbs_to_remove_limit     = nbs_to_remove,
                          export_dir_path         = export_dir_path,
                          export_filename         = export_filename)

[2023-03-20 14:47:27.475192] User 1 - id: 617:#neighbors=100
[2023-03-20 14:47:33.162417] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:47:33.164408] User 2 - id: 877:#neighbors=100
[2023-03-20 14:47:38.377458] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:47:38.378432] User 3 - id: 552:#neighbors=100
[2023-03-20 14:47:42.790931] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:47:42.791900] User 4 - id: 462:#neighbors=100
[2023-03-20 14:47:47.843037] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:47:47.844026] User 5 - id: 598:#neighbors=100
[2023-03-20 14:47:52.146489] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:47:52.147488] User 6 - id: 549:#neighbors=100
[2023-03-20 14:47:57.057677] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:47:57.058675] User 7 - id: 606:#neighbors=100
[2023-03-20 14:48:02.083989] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 1

[2023-03-20 14:51:49.038057] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:51:49.039053] User 60 - id: 39:#neighbors=100
[2023-03-20 14:51:53.338975] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:51:53.339973] User 61 - id: 568:#neighbors=100
[2023-03-20 14:51:55.714612] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:51:55.714612] User 62 - id: 410:#neighbors=100
[2023-03-20 14:51:58.692913] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:51:58.693914] User 63 - id: 142:#neighbors=100
[2023-03-20 14:52:03.281278] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:52:03.282278] User 64 - id: 627:#neighbors=100
[2023-03-20 14:52:07.447892] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:52:07.448892] User 65 - id: 747:#neighbors=100
[2023-03-20 14:52:11.486520] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:52:11.487520] User 66 - id: 974:#neighbors=100
[2023-0

[2023-03-20 14:55:41.671396] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:55:41.671396] User 118 - id: 704:#neighbors=100
[2023-03-20 14:55:46.251772] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:55:46.252772] User 119 - id: 252:#neighbors=100
[2023-03-20 14:55:50.482350] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:55:50.482350] User 120 - id: 847:#neighbors=100
[2023-03-20 14:55:55.290599] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:55:55.292604] User 121 - id: 362:#neighbors=100
[2023-03-20 14:55:58.637681] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:55:58.638682] User 122 - id: 597:#neighbors=100
[2023-03-20 14:56:02.930257] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:56:02.931230] User 123 - id: 765:#neighbors=100
[2023-03-20 14:56:07.207778] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:56:07.207778] User 124 - id: 921:#neighbors=100

[2023-03-20 14:59:58.603436] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 14:59:58.604440] User 176 - id: 445:#neighbors=100
[2023-03-20 15:00:02.340306] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 15:00:02.340306] User 177 - id: 222:#neighbors=100
[2023-03-20 15:00:06.827727] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 15:00:06.827727] User 178 - id: 237:#neighbors=100
[2023-03-20 15:00:11.133297] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 15:00:11.133297] User 179 - id: 845:#neighbors=100
[2023-03-20 15:00:15.433835] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 15:00:15.434834] User 180 - id: 374:#neighbors=100
[2023-03-20 15:00:20.725771] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 15:00:20.726770] User 181 - id: 844:#neighbors=100
[2023-03-20 15:00:25.289191] 10 neighbors removed (refreshed per 10)
----------
[2023-03-20 15:00:25.290190] User 182 - id: 209:#neighbors=100

# Group-level explanations

---

## Experiment 0

In [23]:
bias_lst, pop_lst, expl_size = [], [], []

synth_dir_path = '../datasets/synthetic'
synth_filenames = [filename for filename in os.listdir(synth_dir_path) if filename.split('.')[-1] == 'edges']
for s in synth_filenames:
    filename_no_ext = '.'.join(s.split('.')[:-1])

    info_dict, R = sg.read_graph(synth_dir_path, '.'.join(s.split('.')[:-1])) # Read synthetic dataset
    
    users_num, items_num = info_dict['num_users'], info_dict['num_items']
    num_items_0, users_num_0 = round(info_dict['r_i']*items_num), round(info_dict['r_u']*users_num)
    user_original_ids, item_original_ids = np.arange(users_num), np.arange(items_num)
    item_group_mapped_ids_0, item_group_mapped_ids_1 = np.arange(num_items_0), np.arange(num_items_0, items_num)
    
    u_ids_0 = np.arange(users_num_0)
    i_ids_0 = np.arange(num_items_0, items_num)
    i_ids_1 = np.arange(num_items_0)
    
    P = create_recwalk_model(R)
    Q = calculate_matrix_Q(P)
    ppr_u_to_i = Q[:users_num, users_num:]
    ppr_u_to_u = Q[:users_num, :users_num]
    ppr_i = Q[users_num:]
    
    cur_bias, cur_pop = float(info_dict['p00']), float(info_dict['zipf_a_0'])
    print(f"# {cur_bias}, {cur_pop}")
#     p_U_0_I_0 = np.mean(np.sum(ppr_u_to_i[u_ids_0[:, np.newaxis], i_ids_0], axis=1))
#     p_U_0_I_1 = np.mean(np.sum(ppr_u_to_i[u_ids_0[:, np.newaxis], i_ids_1], axis=1))
    
    # Calculate all deltas
    deltas_0, u_ids_x, i_ids_y = np.array([]), np.array([], dtype=int), np.array([], dtype=int)
    
    num_total_deltas = np.sum(R[u_ids_0[:, np.newaxis], i_ids_1])
    # Keep time started
    start_wall_t = time.time()
    for cur_u_id in u_ids_0:
        cur_u_nbs = np.nonzero(R[cur_u_id, :])[0]
        cur_u_nbs_in_I_1 = np.intersect1d(cur_u_nbs, i_ids_1)
        
        d_term_1 = np.mean(ppr_u_to_u[u_ids_0, cur_u_id])
        for nb_id in cur_u_nbs_in_I_1:
            d_term_2 = calculate_lambda_with_Q_scores(scores=ppr_i,
                                                      x_param=cur_u_id,
                                                      i_param=i_ids_0+users_num,
                                                      S_x=np.array([nb_id]),
                                                      R_x=cur_u_nbs[:, np.newaxis])
            
            if d_term_2 <= 0: continue

            deltas_0 = np.append(deltas_0, d_term_1*d_term_2)
            u_ids_x = np.append(u_ids_x, cur_u_id)
            i_ids_y = np.append(i_ids_y, nb_id)
            
            if deltas_0.shape[0] % 5000 == 0:
                print(f"[{datetime.datetime.now()}] {deltas_0.shape[0]}/{num_total_deltas} deltas calculated.")
    
    sort_inds = np.argsort(deltas_0)[::-1]
    
    # Perform binary search
    low, high = 0, sort_inds.shape[0]
    while True:
        cur_expl_size = (high + low)//2
        print(f"high: {high}, low: {low}")
        print(cur_expl_size)
        
        # Create adjacency matrix
        adj_mat = np.zeros((users_num + items_num, users_num + items_num), dtype=int)
        adj_mat[:users_num, users_num:] = R
        adj_mat[users_num:, :users_num] = R.T
        
        inds_to_remove = sort_inds[:cur_expl_size]
        
        adj_mat[u_ids_x[inds_to_remove], users_num + i_ids_y[inds_to_remove]] = 0
        
        P = create_recwalk_model_from_adj(adj_mat)
        ppr_u_to_i_new = calculate_matrix_Q(P)[:users_num, users_num:]
        
        p_U_0_I_0 = np.mean(np.sum(ppr_u_to_i_new[u_ids_0[:, np.newaxis], i_ids_0], axis=1))
        p_U_0_I_1 = np.mean(np.sum(ppr_u_to_i_new[u_ids_0[:, np.newaxis], i_ids_1], axis=1))
        
        cur_ratio = p_U_0_I_0/p_U_0_I_1
        
        # Binary search step
        if cur_ratio > 1.02:
            high = cur_expl_size
        elif cur_ratio < 1:
            low = cur_expl_size
        else:
            break
            
    bias_lst.append(cur_bias)
    pop_lst.append(cur_pop)
    expl_size.append(cur_expl_size)
    
    print(bias_lst)
    print(pop_lst)
    print(expl_size)
    print(cur_ratio)
    print('----')
    
    # Keep time ended
    end_wall_t = time.time()
    print(f"Wall time: {round((end_wall_t-start_wall_t)/60, 4)} minutes")

# 0.5, 1.1
[2023-04-02 18:23:07.177630] 5000/25170 deltas calculated.
[2023-04-02 18:23:15.777767] 10000/25170 deltas calculated.
[2023-04-02 18:23:26.127906] 15000/25170 deltas calculated.
[2023-04-02 18:23:34.742067] 20000/25170 deltas calculated.
[2023-04-02 18:23:42.722525] 25000/25170 deltas calculated.
high: 25170, low: 0
12585
high: 12585, low: 0
6292
high: 6292, low: 0
3146
high: 3146, low: 0
1573
high: 1573, low: 0
786
high: 786, low: 0
393
[0.5]
[1.1]
[393]
1.0121340857140115
----
Wall time: 1.0789 minutes
# 0.5, 1.3
[2023-04-02 18:24:17.085085] 5000/24887 deltas calculated.
[2023-04-02 18:24:25.280448] 10000/24887 deltas calculated.
[2023-04-02 18:24:35.740566] 15000/24887 deltas calculated.
[2023-04-02 18:24:43.645063] 20000/24887 deltas calculated.
high: 24887, low: 0
12443
high: 12443, low: 0
6221
high: 6221, low: 0
3110
high: 3110, low: 0
1555
high: 1555, low: 0
777
high: 777, low: 0
388
high: 388, low: 0
194
[0.5, 0.5]
[1.1, 1.3]
[393, 194]
1.0158810861420753
----
Wall 

# 0.7, 1.7
[2023-04-02 18:37:33.189845] 5000/35039 deltas calculated.
[2023-04-02 18:37:40.138885] 10000/35039 deltas calculated.
[2023-04-02 18:37:47.369768] 15000/35039 deltas calculated.
[2023-04-02 18:37:54.398819] 20000/35039 deltas calculated.
[2023-04-02 18:38:01.571807] 25000/35039 deltas calculated.
[2023-04-02 18:38:09.944037] 30000/35039 deltas calculated.
[2023-04-02 18:38:17.580722] 35000/35039 deltas calculated.
high: 35039, low: 0
17519
high: 17519, low: 0
8759
high: 8759, low: 0
4379
high: 8759, low: 4379
6569
high: 8759, low: 6569
7664
[0.5, 0.5, 0.5, 0.5, 0.5, 0.6, 0.6, 0.6, 0.6, 0.6, 0.7, 0.7, 0.7, 0.7]
[1.1, 1.3, 1.5, 1.7, 1.0, 1.1, 1.3, 1.5, 1.7, 1.0, 1.1, 1.3, 1.5, 1.7]
[393, 194, 389, 195, 391, 3300, 2821, 3296, 2802, 2826, 7130, 7144, 7653, 7664]
1.0096881763377865
----
Wall time: 1.0929 minutes
# 0.7, 1.0
[2023-04-02 18:38:45.196094] 5000/35046 deltas calculated.
[2023-04-02 18:38:52.092194] 10000/35046 deltas calculated.
[2023-04-02 18:38:58.967305] 15000/3504

# 0.9, 1.5
[2023-04-02 18:49:41.068411] 5000/45021 deltas calculated.
[2023-04-02 18:49:48.615143] 10000/45021 deltas calculated.
[2023-04-02 18:49:55.989964] 15000/45021 deltas calculated.
[2023-04-02 18:50:03.610661] 20000/45021 deltas calculated.
[2023-04-02 18:50:11.111418] 25000/45021 deltas calculated.
[2023-04-02 18:50:18.902013] 30000/45021 deltas calculated.
[2023-04-02 18:50:27.957886] 35000/45021 deltas calculated.
[2023-04-02 18:50:37.811748] 40000/45021 deltas calculated.
high: 41251, low: 0
20625
high: 41251, low: 20625
30938
high: 30938, low: 20625
25781
high: 25781, low: 20625
23203
[0.5, 0.5, 0.5, 0.5, 0.5, 0.6, 0.6, 0.6, 0.6, 0.6, 0.7, 0.7, 0.7, 0.7, 0.7, 0.8, 0.8, 0.8, 0.8, 0.8, 0.9, 0.9, 0.9]
[1.1, 1.3, 1.5, 1.7, 1.0, 1.1, 1.3, 1.5, 1.7, 1.0, 1.1, 1.3, 1.5, 1.7, 1.0, 1.1, 1.3, 1.5, 1.7, 1.0, 1.1, 1.3, 1.5]
[393, 194, 389, 195, 391, 3300, 2821, 3296, 2802, 2826, 7130, 7144, 7653, 7664, 5748, 13127, 13059, 14328, 13542, 10018, 22336, 23006, 23203]
1.0029096155525934
-

In [11]:
def calculate_lambda_with_Q_scores(scores,
                                   x_param,
                                   i_param,
                                   S_x,
                                   R_x,
                                   gamma=0.15):
    # Calculate numerator
    numerator_avg_1 = np.sum(scores[R_x, i_param])/R_x.shape[0]
    numerator_avg_2 = np.sum(scores[S_x, i_param])/S_x.shape[0]
    numerator = ((1-gamma)/gamma)*(numerator_avg_1 - numerator_avg_2)
    
    # Calculate denominator
    denominator_avg_1 = np.sum(scores[R_x, x_param])/R_x.shape[0]
    denominator_avg_2 = np.sum(scores[S_x, x_param])/S_x.shape[0]
    denominator = (R_x.shape[0] - S_x.shape[0])/S_x.shape[0]
    denominator -= ((1-gamma)/gamma)*(denominator_avg_1 - denominator_avg_2)
    
    return numerator/denominator

## Experiment 4

In [5]:
sample_size = 20
k_nbs = 20
nbs_per_round = 20
genre_absorbing_col = 1
export_dir_path = './output/full/group_group/exp4'

synth_dir_path = '../datasets/synthetic'
synth_filenames = [filename for filename in os.listdir(synth_dir_path) if filename.split('.')[-1] == 'edges']

for s in synth_filenames:
    filename_no_ext = '.'.join(s.split('.')[:-1])

    info_dict, R = sg.read_graph(synth_dir_path, '.'.join(s.split('.')[:-1])) # Read synthetic dataset
    
    users_num, items_num = info_dict['num_users'], info_dict['num_items']
    num_items_0 = round(info_dict['r_i']*items_num)
    users_num_0 = round(info_dict['r_u']*users_num)
    user_original_ids, item_original_ids = np.arange(users_num), np.arange(items_num)
    male_mapped_ids = np.arange(users_num_0)
    item_group_mapped_ids_0, item_group_mapped_ids_1 = np.arange(num_items_0), np.arange(num_items_0, items_num)
    
    # Find most central users in group
    P = create_recwalk_model(R)
    Q_u_to_u = calculate_matrix_Q(P)[:users_num, :users_num]
    p_U_0_u_males = np.mean(Q_u_to_u[male_mapped_ids, :], axis=0)[male_mapped_ids]
    
    sample_users = male_mapped_ids[np.argsort(p_U_0_u_males)[::-1]][:sample_size]
    
    export_filename = f"exp4_{sample_size}_{k_nbs}_{filename_no_ext}"
    
    print(f"Experiment for dataset {s} started..")
    remove_best_k(R                       = R,
                  user_original_ids       = user_original_ids,
                  item_original_ids       = item_original_ids,
                  user_group_mapped_ids   = male_mapped_ids,
                  item_group_mapped_ids_0 = item_group_mapped_ids_0,
                  item_group_mapped_ids_1 = item_group_mapped_ids_1,
                  sample_users            = sample_users,
                  k_nbs                   = k_nbs,
                  genre_absorbing_col     = genre_absorbing_col,
                  nbs_per_round           = nbs_per_round,
                  export_dir_path         = export_dir_path,
                  export_filename         = export_filename)
    print(f"Experiment ended..")

Experiment for dataset synth_0.5_b_1.1_p.edges started..
[2023-03-20 15:25:41.066496] User 1 - id: 313:#neighbors=100
[2023-03-20 15:25:42.372754] User 2 - id: 397:#neighbors=100
[2023-03-20 15:25:43.634019] User 3 - id: 275:#neighbors=100
[2023-03-20 15:25:45.062202] User 4 - id: 377:#neighbors=100
[2023-03-20 15:25:46.574343] User 5 - id: 281:#neighbors=100
[2023-03-20 15:25:47.926568] User 6 - id: 333:#neighbors=100
[2023-03-20 15:25:49.198840] User 7 - id: 192:#neighbors=100
[2023-03-20 15:25:50.486131] User 8 - id: 299:#neighbors=100
[2023-03-20 15:25:51.741384] User 9 - id: 74:#neighbors=100
[2023-03-20 15:25:53.045631] User 10 - id: 351:#neighbors=100
[2023-03-20 15:25:54.499797] User 11 - id: 177:#neighbors=100
[2023-03-20 15:25:56.031920] User 12 - id: 215:#neighbors=100
[2023-03-20 15:25:57.522073] User 13 - id: 20:#neighbors=100
[2023-03-20 15:25:59.056196] User 14 - id: 290:#neighbors=100
[2023-03-20 15:26:00.377430] User 15 - id: 305:#neighbors=100
[2023-03-20 15:26:01.644

[2023-03-20 15:31:39.698851] User 19 - id: 405:#neighbors=100
[2023-03-20 15:31:40.917118] User 20 - id: 150:#neighbors=100
[2023-03-20 15:31:49.866999] Round 1/20 completed.
[2023-03-20 15:31:53.590861] Round 2/20 completed.
[2023-03-20 15:31:57.709501] Round 3/20 completed.
[2023-03-20 15:32:00.002492] Round 4/20 completed.
[2023-03-20 15:32:03.736355] Round 5/20 completed.
[2023-03-20 15:32:07.478248] Round 6/20 completed.
[2023-03-20 15:32:11.110131] Round 7/20 completed.
[2023-03-20 15:32:14.778039] Round 8/20 completed.
[2023-03-20 15:32:18.494912] Round 9/20 completed.
[2023-03-20 15:32:22.165799] Round 10/20 completed.
[2023-03-20 15:32:26.004603] Round 11/20 completed.
[2023-03-20 15:32:29.933352] Round 12/20 completed.
[2023-03-20 15:32:33.607247] Round 13/20 completed.
[2023-03-20 15:32:37.519014] Round 14/20 completed.
[2023-03-20 15:32:41.740599] Round 15/20 completed.
[2023-03-20 15:32:45.611374] Round 16/20 completed.
[2023-03-20 15:32:49.445178] Round 17/20 completed.
[

Experiment for dataset synth_0.6_b_1.5_p.edges started..
[2023-03-20 15:38:51.319724] User 1 - id: 120:#neighbors=100
[2023-03-20 15:38:52.611012] User 2 - id: 318:#neighbors=100
[2023-03-20 15:38:54.131106] User 3 - id: 170:#neighbors=100
[2023-03-20 15:38:55.554291] User 4 - id: 110:#neighbors=100
[2023-03-20 15:38:57.092447] User 5 - id: 498:#neighbors=100
[2023-03-20 15:38:58.546583] User 6 - id: 411:#neighbors=100
[2023-03-20 15:38:59.962766] User 7 - id: 486:#neighbors=100
[2023-03-20 15:39:01.265020] User 8 - id: 126:#neighbors=100
[2023-03-20 15:39:02.570309] User 9 - id: 219:#neighbors=100
[2023-03-20 15:39:03.808573] User 10 - id: 331:#neighbors=100
[2023-03-20 15:39:05.084840] User 11 - id: 453:#neighbors=100
[2023-03-20 15:39:06.416071] User 12 - id: 204:#neighbors=100
[2023-03-20 15:39:07.926208] User 13 - id: 479:#neighbors=100
[2023-03-20 15:39:10.129521] User 14 - id: 324:#neighbors=100
[2023-03-20 15:39:11.676670] User 15 - id: 239:#neighbors=100
[2023-03-20 15:39:12.8

[2023-03-20 15:44:56.573376] User 19 - id: 105:#neighbors=100
[2023-03-20 15:44:58.160467] User 20 - id: 250:#neighbors=100
[2023-03-20 15:45:07.064369] Round 1/20 completed.
[2023-03-20 15:45:10.720269] Round 2/20 completed.
[2023-03-20 15:45:14.388167] Round 3/20 completed.
[2023-03-20 15:45:17.827201] Round 4/20 completed.
[2023-03-20 15:45:20.147866] Round 5/20 completed.
[2023-03-20 15:45:23.796785] Round 6/20 completed.
[2023-03-20 15:45:27.433697] Round 7/20 completed.
[2023-03-20 15:45:31.045627] Round 8/20 completed.
[2023-03-20 15:45:34.834465] Round 9/20 completed.
[2023-03-20 15:45:38.724233] Round 10/20 completed.
[2023-03-20 15:45:42.468085] Round 11/20 completed.
[2023-03-20 15:45:46.461805] Round 12/20 completed.
[2023-03-20 15:45:50.118740] Round 13/20 completed.
[2023-03-20 15:45:52.798171] Round 14/20 completed.
[2023-03-20 15:45:54.790028] Round 15/20 completed.
[2023-03-20 15:45:56.527035] Round 16/20 completed.
[2023-03-20 15:45:57.683373] Round 17/20 completed.
[

Experiment for dataset synth_0.7_b_1_p.edges started..
[2023-03-20 15:51:37.671132] User 1 - id: 346:#neighbors=100
[2023-03-20 15:51:38.947401] User 2 - id: 198:#neighbors=100
[2023-03-20 15:51:40.178696] User 3 - id: 460:#neighbors=100
[2023-03-20 15:51:41.414988] User 4 - id: 425:#neighbors=100
[2023-03-20 15:51:42.654314] User 5 - id: 62:#neighbors=100
[2023-03-20 15:51:43.891569] User 6 - id: 58:#neighbors=100
[2023-03-20 15:51:45.339748] User 7 - id: 305:#neighbors=100
[2023-03-20 15:51:46.768921] User 8 - id: 27:#neighbors=100
[2023-03-20 15:51:48.006213] User 9 - id: 121:#neighbors=100
[2023-03-20 15:51:49.228520] User 10 - id: 497:#neighbors=100
[2023-03-20 15:51:50.460807] User 11 - id: 272:#neighbors=100
[2023-03-20 15:51:51.721093] User 12 - id: 375:#neighbors=100
[2023-03-20 15:51:52.958385] User 13 - id: 266:#neighbors=100
[2023-03-20 15:51:54.370575] User 14 - id: 334:#neighbors=100
[2023-03-20 15:51:55.789757] User 15 - id: 308:#neighbors=100
[2023-03-20 15:51:57.322877

[2023-03-20 15:57:20.145158] User 19 - id: 231:#neighbors=100
[2023-03-20 15:57:21.370484] User 20 - id: 130:#neighbors=100
[2023-03-20 15:57:30.098490] Round 1/20 completed.
[2023-03-20 15:57:33.787385] Round 2/20 completed.
[2023-03-20 15:57:37.249396] Round 3/20 completed.
[2023-03-20 15:57:40.897310] Round 4/20 completed.
[2023-03-20 15:57:44.223408] Round 5/20 completed.
[2023-03-20 15:57:48.231110] Round 6/20 completed.
[2023-03-20 15:57:50.319907] Round 7/20 completed.
[2023-03-20 15:57:53.306200] Round 8/20 completed.
[2023-03-20 15:57:57.456761] Round 9/20 completed.
[2023-03-20 15:58:01.343532] Round 10/20 completed.
[2023-03-20 15:58:04.620657] Round 11/20 completed.
[2023-03-20 15:58:08.252613] Round 12/20 completed.
[2023-03-20 15:58:11.835532] Round 13/20 completed.
[2023-03-20 15:58:15.860223] Round 14/20 completed.
[2023-03-20 15:58:17.787114] Round 15/20 completed.
[2023-03-20 15:58:20.570558] Round 16/20 completed.
[2023-03-20 15:58:24.436308] Round 17/20 completed.
[

Experiment for dataset synth_0.9_b_1.3_p.edges started..
[2023-03-20 16:03:52.530075] User 1 - id: 410:#neighbors=100
[2023-03-20 16:03:53.888297] User 2 - id: 13:#neighbors=100
[2023-03-20 16:03:55.268515] User 3 - id: 448:#neighbors=100
[2023-03-20 16:03:56.712689] User 4 - id: 495:#neighbors=100
[2023-03-20 16:03:58.155856] User 5 - id: 286:#neighbors=100
[2023-03-20 16:03:59.650005] User 6 - id: 64:#neighbors=100
[2023-03-20 16:04:00.921305] User 7 - id: 393:#neighbors=100
[2023-03-20 16:04:02.139571] User 8 - id: 287:#neighbors=100
[2023-03-20 16:04:03.360872] User 9 - id: 432:#neighbors=100
[2023-03-20 16:04:04.651141] User 10 - id: 0:#neighbors=100
[2023-03-20 16:04:05.892422] User 11 - id: 359:#neighbors=100
[2023-03-20 16:04:07.087773] User 12 - id: 60:#neighbors=100
[2023-03-20 16:04:08.286051] User 13 - id: 318:#neighbors=100
[2023-03-20 16:04:09.353439] User 14 - id: 438:#neighbors=100
[2023-03-20 16:04:10.146989] User 15 - id: 99:#neighbors=100
[2023-03-20 16:04:11.333306]

[2023-03-20 16:09:37.558155] User 19 - id: 168:#neighbors=100
[2023-03-20 16:09:38.767460] User 20 - id: 495:#neighbors=100
[2023-03-20 16:09:47.983192] Round 1/20 completed.
[2023-03-20 16:09:51.650082] Round 2/20 completed.
[2023-03-20 16:09:55.662786] Round 3/20 completed.
[2023-03-20 16:09:59.577550] Round 4/20 completed.
[2023-03-20 16:10:03.516287] Round 5/20 completed.
[2023-03-20 16:10:07.274142] Round 6/20 completed.
[2023-03-20 16:10:10.958032] Round 7/20 completed.
[2023-03-20 16:10:14.507993] Round 8/20 completed.
[2023-03-20 16:10:18.099935] Round 9/20 completed.
[2023-03-20 16:10:21.654901] Round 10/20 completed.
[2023-03-20 16:10:25.274834] Round 11/20 completed.
[2023-03-20 16:10:28.938078] Round 12/20 completed.
[2023-03-20 16:10:32.446070] Round 13/20 completed.
[2023-03-20 16:10:36.102975] Round 14/20 completed.
[2023-03-20 16:10:39.926778] Round 15/20 completed.
[2023-03-20 16:10:43.528719] Round 16/20 completed.
[2023-03-20 16:10:46.653934] Round 17/20 completed.
[

### Experiment 2

In [3]:
# Constants
genre_absorbing_col = 1
users_group_is_male = True
directions_removed = 1
sample_from_user_group = True

# Read dataset
synth_directory = '../datasets/synthetic'
filename = 'synthetic_2_2'
filepath = os.path.join(synth_directory, filename + '.tsv')

suffix_0 = filename.split('_')[1]
suffix_1 = filename.split('_')[2]
    
# Read synthetic dataset
metadata, R = read_synthetic_dataset_from_file(filepath)
users_num, items_num = metadata['num_users'], metadata['num_items']

# Create dataset object
num_users_0 = round(metadata['r_u']*users_num)
num_items_0 = round(metadata['r_i']*items_num)
user_ids_U_0 = np.arange(num_users_0)
item_ids_I_0 = np.arange(num_items_0)
genres_str = ['Action', 'Romance']

my_dataset = UserItemDataset()
my_dataset.load_synthetic_data(R,
                               user_ids_U_0,
                               item_ids_I_0,
                               genres_str)

print(f"[{datetime.datetime.now()}] --- Synthetic dataset '{filename}' has been loaded..")

user_original_ids, item_original_ids = np.arange(users_num), np.arange(items_num)

# Parameter variables
# sample_size_list = [5, 10, 20, 40, 80, 400]
# num_best_neighbors_list = [80, 40, 20, 10, 5, 1]
sample_size_list = [200]
num_best_neighbors_list = [2]
export_dir_path = './output/group/experiment2'
for i in range(len(sample_size_list)):
    sample_size = sample_size_list[i]
    num_best_neighbors = num_best_neighbors_list[i]
    export_filename = f"experiment1_s_{'males' if sample_from_user_group else 'both'}_{sample_size}_{num_best_neighbors}"
    
    print(f"[{datetime.datetime.now()}] --- Experiment for values ({sample_size},{num_best_neighbors}) started..")
    group_default_experiment(sample_size=sample_size,
                             num_best_neighbors=num_best_neighbors,
                             genre_absorbing_col=genre_absorbing_col,
                             users_group_is_male=users_group_is_male,
                             absorbing_genres_str=genres_str,
                             directions_removed=directions_removed,
                             R=R,
                             user_original_ids=user_original_ids,
                             item_original_ids=item_original_ids,
                             my_dataset=my_dataset,
                             sample_from_user_group=sample_from_user_group,
                             export_filename=export_filename,
                             export_dir_path=export_dir_path)
    print(f"[{datetime.datetime.now()}] --- Experiment ended..")

[2023-01-31 19:17:55.689589] --- Synthetic dataset 'synthetic_2_2' has been loaded..
[2023-01-31 19:17:55.692588] --- Experiment for values (200,2) started..
[2023-01-31 19:18:00.872622] Users with greatest PPR score AND from group are selected..
[2023-01-31 19:18:00.881615] Minimum number of neighbors: 100
[2023-01-31 19:18:04.103769] --- Delta scores calculation started..
[2023-01-31 19:23:30.395678] --- Delta scores calculation ended..
[2023-01-31 19:23:30.402673] --- Best neighbor removal started..
[2023-01-31 19:23:30.402673] (Info) Neighbors to be removed: 20 (per round)
[2023-01-31 19:24:08.829675] Round 5/20 ended.
[2023-01-31 19:24:48.085290] Round 10/20 ended.
[2023-01-31 19:25:26.474325] Round 15/20 ended.
[2023-01-31 19:26:06.411754] Round 20/20 ended.
Wall time: 8.0385 minutes
[2023-01-31 19:26:14.921843] --- Experiment ended..


### Experiment 3

In [4]:
##### Experiment paramaters #####
sample_size = 20
num_best_neighbors = 20
genre_absorbing_col = 1
users_group_is_male = True
item_group_genre = 'Romance'
directions_removed = 1

sample_from_user_group = True
#################################

synth_directory = '../datasets/synthetic'
filename = 'synthetic_2_2'
filepath = os.path.join(synth_directory, filename + '.tsv')

suffix_0 = filename.split('_')[1]
suffix_1 = filename.split('_')[2]
    
# Read synthetic dataset
metadata, R = read_synthetic_dataset_from_file(filepath)
users_num, items_num = metadata['num_users'], metadata['num_items']

# Create dataset object
num_users_0 = round(metadata['r_u']*users_num)
num_items_0 = round(metadata['r_i']*items_num)
user_ids_U_0 = np.arange(num_users_0)
item_ids_I_0 = np.arange(num_items_0)
genres_str = ['Action', 'Romance']

my_dataset = UserItemDataset()
my_dataset.load_synthetic_data(R,
                               user_ids_U_0,
                               item_ids_I_0,
                               genres_str)

print(f"[{datetime.datetime.now()}] --- Synthetic dataset '{filename}' has been loaded..")

user_original_ids, item_original_ids = np.arange(users_num), np.arange(items_num)

export_dir_path = './output/group/experiment3'
export_filename = f"experiment3_s{'_males' if sample_from_user_group else '_all'}_{suffix_0}_{suffix_1}_{sample_size}_{num_best_neighbors}"

group_experiment_3(sample_size=sample_size,
                   num_best_neighbors=num_best_neighbors,
                   genre_absorbing_col=genre_absorbing_col,
                   users_group_is_male=users_group_is_male,
                   item_group_genre=item_group_genre,
                   directions_removed=directions_removed,
                   R=R,
                   user_original_ids=user_original_ids,
                   item_original_ids=item_original_ids,
                   my_dataset=my_dataset,
                   sample_from_user_group=sample_from_user_group,
                   export_dir_path=export_dir_path,
                   export_filename=export_filename)

[2023-01-31 20:38:19.051821] --- Synthetic dataset 'synthetic_2_2' has been loaded..
[2023-01-31 20:38:22.392910] Users with greatest PPR score AND from group are selected..
[2023-01-31 20:38:22.403901] Minimum number of neighbors: 100
[2023-01-31 20:38:24.060950] --- Delta scores calculation started..
[2023-01-31 20:50:52.947382] --- Delta scores calculation ended..
Wall time: 12.4817 minutes


### Experiment 4

In [3]:
##### Experiment paramaters #####
sample_size = 20
num_best_neighbors = 20

genre_absorbing_col = 1
users_group_is_male = True
sample_from_user_group = True
directions_removed = 1
#################################

synth_directory = '../datasets/synthetic'
synthetic_filenames = os.listdir(synth_directory)

export_dir_path = './output/group/experiment4'

for filename in synthetic_filenames:
    # Split synthetic dataset suffixes
    suffix_0 = filename.split('.')[0].split('_')[1]
    suffix_1 = filename.split('.')[0].split('_')[2]
    export_filename = f"experiment1_s_{'males' if sample_from_user_group else 'both'}_{suffix_0}_{suffix_1}_{sample_size}_{num_best_neighbors}"
    
    # Read synthetic dataset
    filepath = os.path.join(synth_directory, filename)
    metadata, R = read_synthetic_dataset_from_file(filepath)
    users_num, items_num = metadata['num_users'], metadata['num_items']
    
    # Create dataset object
    num_users_0 = round(metadata['r_u']*users_num)
    num_items_0 = round(metadata['r_i']*items_num)
    user_ids_U_0 = np.arange(num_users_0)
    item_ids_I_0 = np.arange(num_items_0)

    genres_str = ['Action', 'Romance']

    my_dataset = UserItemDataset()
    my_dataset.load_synthetic_data(R,
                                   user_ids_U_0,
                                   item_ids_I_0,
                                   genres_str)
    
    print(f"[{datetime.datetime.now()}] --- Synthetic dataset '{filename}' has been loaded..")

    user_original_ids, item_original_ids = np.arange(users_num), np.arange(items_num)
    
    # Start experiment
    print(f"[{datetime.datetime.now()}] --- Experiment for values ({sample_size},{num_best_neighbors}) started..")
    group_default_experiment(sample_size            = sample_size,
                             num_best_neighbors     = num_best_neighbors,
                             genre_absorbing_col    = genre_absorbing_col,
                             users_group_is_male    = users_group_is_male,
                             absorbing_genres_str   = genres_str,
                             directions_removed     = directions_removed,
                             R                      = R,
                             user_original_ids      = user_original_ids,
                             item_original_ids      = item_original_ids,
                             my_dataset             = my_dataset,
                             sample_from_user_group = sample_from_user_group,
                             export_filename        = export_filename,
                             export_dir_path        = export_dir_path)
    print(f"[{datetime.datetime.now()}] --- Experiment ended..")

[2023-01-31 19:39:40.575928] --- Synthetic dataset 'synthetic_0_0.tsv' has been loaded..
[2023-01-31 19:39:40.576926] --- Experiment for values (20,20) started..
[2023-01-31 19:39:44.387744] Users with greatest PPR score AND from group are selected..
Sample user original ids:
478	 210	 318	 246	 54	 227	 76	 284	 129	 116	 108	 44	 114	 496	 166	 138	 249	 377	 497	 191	 
[2023-01-31 19:39:44.389742] --- Experiment ended..
[2023-01-31 19:39:46.275662] --- Synthetic dataset 'synthetic_0_1.tsv' has been loaded..
[2023-01-31 19:39:46.276663] --- Experiment for values (20,20) started..
[2023-01-31 19:39:50.302359] Users with greatest PPR score AND from group are selected..
Sample user original ids:
418	 207	 327	 375	 465	 290	 213	 31	 449	 132	 153	 145	 11	 69	 412	 314	 286	 377	 328	 140	 
[2023-01-31 19:39:50.310361] --- Experiment ended..
[2023-01-31 19:39:51.920442] --- Synthetic dataset 'synthetic_0_2.tsv' has been loaded..
[2023-01-31 19:39:51.920442] --- Experiment for values (2

[2023-01-31 19:41:44.898740] Users with greatest PPR score AND from group are selected..
Sample user original ids:
235	 300	 437	 46	 322	 96	 368	 312	 43	 429	 24	 395	 197	 50	 398	 233	 347	 356	 32	 449	 
[2023-01-31 19:41:44.902738] --- Experiment ended..
[2023-01-31 19:41:48.960414] --- Synthetic dataset 'synthetic_3_2.tsv' has been loaded..
[2023-01-31 19:41:48.961416] --- Experiment for values (20,20) started..
[2023-01-31 19:41:53.872168] Users with greatest PPR score AND from group are selected..
Sample user original ids:
4	 257	 172	 495	 78	 201	 231	 16	 456	 343	 283	 128	 453	 99	 306	 443	 481	 64	 318	 217	 
[2023-01-31 19:41:53.875167] --- Experiment ended..
[2023-01-31 19:41:57.076921] --- Synthetic dataset 'synthetic_3_3.tsv' has been loaded..
[2023-01-31 19:41:57.076921] --- Experiment for values (20,20) started..
[2023-01-31 19:42:03.700655] Users with greatest PPR score AND from group are selected..
Sample user original ids:
167	 69	 387	 273	 494	 403	 213	 241

## Plot synthetic data histograms

In [ ]:
def plot_synthetic_data_histogram(num_users,
                                  num_items,
                                  r_u,
                                  r_i,
                                  p00,
                                  p11,
                                  ratings_per_user,
                                  zipf_a_0,
                                  zipf_a_1,
                                  synthetic_R):
    # Calculate number of users of each group U0 and U1
    num_users_0 = round(r_u*num_users)
    num_users_1 = num_users - num_users_0

    # Calculate number of items of each group I0 and I1
    num_items_0 = round(r_i*num_items)
    num_items_1 = num_items - num_items_0

    col_sums_I_0 = np.sum(synthetic_R[:, :num_items_0], axis=0)
    col_sums_I_1 = np.sum(synthetic_R[:, num_items_0:], axis=0)

    print(f"#### Synthetic data ####")
    print(f"Parameters: num_users={num_users}, num_items={num_items}, r_u={r_u}, r_i={r_i}, p00={p00}, p11={p11}, " +
           f"ratings per user={ratings_per_user}")
    print("Resulted rates:")
    print(f"Rate of ratings from U_0 to I_0: {np.count_nonzero(synthetic_R[:num_users_0, :num_items_0])/np.count_nonzero(synthetic_R[:num_users_0])}")
    print(f"Rate of ratings from U_0 to I_1: {np.count_nonzero(synthetic_R[:num_users_0, num_items_0:])/np.count_nonzero(synthetic_R[:num_users_0])}")
    print()
    print(f"Rate of ratings from U_1 to I_0: {np.count_nonzero(synthetic_R[num_users_0:, :num_items_0])/np.count_nonzero(synthetic_R[num_users_0:])}")
    print(f"Rate of ratings from U_1 to I_1: {np.count_nonzero(synthetic_R[num_users_0:, num_items_0:])/np.count_nonzero(synthetic_R[num_users_0:])}")

    fig, (ax0, ax1) = plt.subplots(1,2, figsize=(12,6))
    ax0.set_title(f"I_0 (a={zipf_a_0})")
    ax0.set_xlabel('Items (sorted)')
    ax0.set_ylabel('Total I_0 ratings ratio')
    items_0_ratio = [col_sums_I_0[i]/np.count_nonzero(synthetic_R[:, :num_items_0]) for i in range(col_sums_I_0.shape[0])]
    items_0_ratio = np.sort(items_0_ratio)[::-1]
    for i in range(col_sums_I_0.shape[0]):
        ax0.bar(i+1, items_0_ratio[i], color='r')

    ax1.set_title(f"I_1 (a={zipf_a_1})")
    ax1.set_xlabel('Items (sorted)')
    ax1.set_ylabel('Total I_1 ratings ratio')
    items_1_ratio = [col_sums_I_1[i]/np.count_nonzero(synthetic_R[:, num_items_0:]) for i in range(col_sums_I_1.shape[0])]
    items_1_ratio = np.sort(items_1_ratio)[::-1]
    for i in range(col_sums_I_1.shape[0]):
        ax1.bar(i+1, items_1_ratio[i], color='g')

    plt.show()

In [5]:
synth_directory = '../datasets/synthetic'
filename = 'synthetic_3_0'
filepath = os.path.join(synth_directory, filename + '.tsv')

suffix_0 = filename.split('_')[1]
suffix_1 = filename.split('_')[2]
    
# Read synthetic dataset
start = time.time()
metadata, R = read_synthetic_dataset_from_file(filepath)
end = time.time()

print(round(end-start, 4))

1.6089
